# S04 T01: Transformació Registre Log amb Regular expressions

#### David Arroyo

In [67]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open ('Datos\Web_access_log-akumenius.com.txt', 'r') as file:
    lines_list = []
    for line in file:
        line_string = line.strip()
        lines_list.append(line_string)

In [3]:
log_df = pd.DataFrame([lines_list]).T

In [4]:
log_df.shape

(261873, 1)

In [5]:
log_df.rename(columns = {0:'Record'}, inplace = True)

In [6]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261873 entries, 0 to 261872
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Record  261873 non-null  object
dtypes: object(1)
memory usage: 2.0+ MB


In [39]:
'''
Function to verify weather a pattern is found in a column of a data frame. 

Inputs: 
patern:  Regular expression in a string format
df: Dataframe to be used
column: Name of the column where we want to perform the search

Outputs:
Added column named 'Result'. Boolean, True or False 
Added column named 'itemFound'. Item foud or None, in case no item matches the pattern

'''

def serch_pattern (pattern, df, column):
    
    serched_list = df[column].tolist()
    result_list = []
    serched_item_list = []

    for string in serched_list:
        serched_item = re.match(pattern, string)
        
        if serched_item:
            result_list.append (True)
            serched_item_list.append(serched_item.group(0))
        else:
            result_list.append (False)
            serched_item_list.append(None) 
    
    df['Result'] = result_list
    df['itemFound'] = serched_item_list
         

In [43]:
# Test df to validate the different metodologies used to clean and transform the final DF

test_df = log_df.loc[:200, 'Record'].to_frame()


In [44]:
test_df.head()

,Record
0,localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 ...
1,localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 ...
2,localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 ...
3,localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 ...
4,localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 ...


#### Eliminate rows not starting with a web page pattern

In [45]:
web_pattern = '^([a-z0-9][a-z0-9\-]*\.)+([a-z0-9][a-z0-9\-]*)'

In [46]:
serch_pattern (web_pattern,test_df,'Record')

In [49]:
test_df.drop(test_df.loc[test_df['Result']== False].index, inplace=True)

In [50]:
test_df.head()

,Record,Result,itemFound
22,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,True,www.akumenius.com
23,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,True,www.akumenius.com
24,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,True,www.akumenius.com
25,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,True,www.akumenius.com
26,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,True,www.akumenius.com


#### Eliminate 'Result' Column and Rename 'itemFound' Column


In [56]:
test_df.drop(columns = ['Result'], inplace = True)

In [57]:
test_df.rename(columns = {'itemFound': 'WebPage'})

,Record,WebPage
22,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
23,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
24,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
25,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
26,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
...,...,...
196,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
197,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com
198,www.akumenius.com 180.76.5.64 - - [23/Feb/2014...,www.akumenius.com
199,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com


#### New column with IP addresses
Split the Record line into 3 parts separated by white spaces. The second item would match the IP Adresses.

In [59]:
serched_list = test_df['Record'].tolist()

In [60]:
ipAdresses_list = []

for row in serched_list:
    list_row = re.split('\s', row ,2)
    ipAdresses_list.append (list_row [1])

In [62]:
# Add ipAdresses_List to the the df
test_df ['IPAdress'] = ipAdresses_list

In [63]:
test_df.head()

,Record,itemFound,IPAdress
22,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com,66.249.76.216
23,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com,66.249.76.216
24,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com,66.249.76.216
25,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com,66.249.76.216
26,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,www.akumenius.com,66.249.76.216


In [64]:
# Make sure all the obtaided values in the IPAdress column have the IP format. IP Adress are a sequence of 4 numbers,
# separated by dots '.'

ipPattern = '^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$'

serch_pattern (ipPattern, test_df, 'IPAdress')

In [65]:
test_df.head()

,Record,itemFound,IPAdress,Result
22,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,66.249.76.216,66.249.76.216,True
23,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,66.249.76.216,66.249.76.216,True
24,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,66.249.76.216,66.249.76.216,True
25,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,66.249.76.216,66.249.76.216,True
26,www.akumenius.com 66.249.76.216 - - [23/Feb/20...,66.249.76.216,66.249.76.216,True


In [73]:
# Eliminate rows with iligal IP Adress

test_df.drop(test_df.loc[test_df['Result']== False].index, inplace=True)


In [74]:
test_df.drop(columns = ['Result','itemFound'], inplace = True)

#### To separate date from each row

In [77]:
date_list = []

for string in serched_list:
    date_start_position = string.find('[')
    date_end_position = string.find(']')
    date_string = string[date_start_position+1:date_end_position]
    date_list.append(date_string)

In [78]:
date_list

['23/Feb/2014:03:10:31 +0100',
 '23/Feb/2014:03:10:33 +0100',
 '23/Feb/2014:03:10:35 +0100',
 '23/Feb/2014:03:10:38 +0100',
 '23/Feb/2014:03:10:39 +0100',
 '23/Feb/2014:03:10:40 +0100',
 '23/Feb/2014:03:10:42 +0100',
 '23/Feb/2014:03:10:45 +0100',
 '23/Feb/2014:03:10:46 +0100',
 '23/Feb/2014:03:10:47 +0100',
 '23/Feb/2014:03:10:49 +0100',
 '23/Feb/2014:03:10:52 +0100',
 '23/Feb/2014:03:10:53 +0100',
 '23/Feb/2014:03:10:54 +0100',
 '23/Feb/2014:03:10:56 +0100',
 '23/Feb/2014:03:10:56 +0100',
 '23/Feb/2014:03:10:59 +0100',
 '23/Feb/2014:03:11:01 +0100',
 '23/Feb/2014:03:11:01 +0100',
 '23/Feb/2014:03:11:02 +0100',
 '23/Feb/2014:03:11:03 +0100',
 '23/Feb/2014:03:11:06 +0100',
 '23/Feb/2014:03:11:07 +0100',
 '23/Feb/2014:03:11:08 +0100',
 '23/Feb/2014:03:11:09 +0100',
 '23/Feb/2014:03:11:10 +0100',
 '23/Feb/2014:03:11:13 +0100',
 '23/Feb/2014:03:11:14 +0100',
 '23/Feb/2014:03:11:15 +0100',
 '23/Feb/2014:03:11:16 +0100',
 '23/Feb/2014:03:11:17 +0100',
 '23/Feb/2014:03:11:20 +0100',
 '23/Feb

In [ ]:
# Delete rows starting with 'localhost', as they don`t bring any usefull information

test_df = log_df[log_df['Record'].astype(str).str.startswith ('localhost') == True]


list_record = test_df['Record'].tolist()

print(list_record)

In [ ]:
# The structure of the line starts with the web page, folowed by 1 white space, IP Address and 1 white space more. 

record_list = []
for row in list_record:
    list_row = re.split('\s', row ,2)
    record_list.append (list_row)


In [ ]:
# We start building a new DF with usefull and preprocessed columns
webpage = []
IPAddress = []

for i in range (len (record_list)):
    wp = record_list [i][0]
    ip = record_list [i][1]
    webpage.append (wp)
    IPAddress.append (ip)
    

In [ ]:
useful_df = pd.DataFrame ({'webpage': webpage,
                           'IPAddress': IPAddress})

In [ ]:
useful_df.head(50)